In [1]:
# bob.refresh {"cron": "5,30 2-12 * * *", "timeout": 40000}

In [2]:
%run __init__.ipynb

Successfully connected to MongoDB


In [3]:
#Highlighted KPIS
def domain_108(df):
    domain = df.copy()
    #Calcul US
    domain = domain.groupby('Date', as_index=False).agg({'Confirmed':'sum', 'Deaths':'sum' , 'Population':'sum'})
    domain.loc[(domain['Deaths'] != 0),'Fatality Rate'] = round((domain['Deaths']/domain['Confirmed']*100),2)
    domain.loc[(domain['Confirmed'] != 0),'Confirmed Ratio'] = round((domain['Confirmed']/domain['Population']*100000),0)
    domain.loc[(domain['Deaths'] != 0),'Deaths Ratio'] = round((domain['Deaths']/domain['Population']*100000),0)
    
    #Create DF for HKPIS
    cols_to_keep = ['Date']
    domain= domain.melt(id_vars=cols_to_keep,value_name='VALUE',var_name='KPI')
    
    #Calc variation
    kpis = domain['KPI'].drop_duplicates().values.tolist()
    final_domain = pd.DataFrame()
    for kpi in kpis:
        tmp_df = domain[domain['KPI'] == kpi].sort_values(by="Date", ascending=False).reset_index(drop=True)
        final_domain = final_domain.append(tmp_df)
    
    #Create df last + merge
    final_domain['D'] = (final_domain.index).astype(str)
    final_domain['D-1'] = (final_domain.index + 1).astype(str)
    cols_to_rename = {"D": "D-1",'VALUE':'VALUE_D-1'}
    df_last = final_domain[['D','KPI','VALUE']].rename(index=str, columns=cols_to_rename) 
    final_domain = final_domain.merge(df_last, on= ['D-1','KPI'], how= 'left').drop(['D','D-1'], axis=1)
    
    #Add fields 
    #SCENARIO
    final_domain['SCENARIO'] = pd.to_datetime(final_domain['Date'], format='%m/%d/%y').dt.strftime('%d/%m/%Y')
    
    #VARIATION
    final_domain['VAR'] = final_domain['VALUE'] - final_domain['VALUE_D-1']
    final_domain['VARP'] = final_domain['VAR'] / abs(final_domain['VALUE_D-1']) * 100
    
    #UNIT & PRECISION
    final_domain.loc[:,'UNIT_VALUE'] = ''
    final_domain.loc[final_domain['KPI'] == 'Fatality Rate','UNIT_VALUE'] = ' %'
    final_domain.loc[:,'UNIT_VAR'] = ''
    final_domain.loc[final_domain['KPI'] == 'Fatality Rate','UNIT_VAR'] = ' pts'
    final_domain.loc[:,'PRECISION_VALUE'] = ',.0f'
    final_domain.loc[final_domain['KPI'] == 'Fatality Rate','PRECISION_VALUE'] = ',.2f'
    final_domain.loc[:,'PRECISION_VAR'] = '+,.0f'
    final_domain.loc[final_domain['KPI'] == 'Fatality Rate','PRECISION_VAR'] = '+,.3f'
    
    final_domain = final_domain.drop('Date',axis=1).dropna().reset_index(drop=True)
    if not final_domain.empty:
        bob.mongo.save_df(final_domain, '108', DB_APP, True)
    return final_domain

In [4]:
%%time

# Variables
bob_statics = ''

# Load Data Source
def check_data():
    # Data check
    file_path_check = os.path.join(OUTPUT_FOLDER,'US_MAP.csv')
    df_check = pd.DataFrame()
    if path.exists(file_path_check):
        df_check = pd.read_csv(file_path_check, sep=";")
        df_check = df_check[(df_check['Date'] == YESTERDAY.strftime('%d/%m/%y'))].reset_index(drop=True)
        if not df_check.empty:
            print(f'Data already updated !')
            %stop 
    return df_check


# Trasnform Data Source
def transform_data(df):
    #--Create Dataframe
    def create_df(df_name, kpi, agg, cols_to_keep):
        file_path_conf = os.path.join(OUTPUT_FOLDER, f'{df_name}.csv')
        df = pd.read_csv(file_path_conf, sep=";")
        df = df.drop('Unnamed: 0',axis=1)
        df = df.melt(id_vars=cols_to_keep,value_name=kpi,var_name='Date')
        df = df.groupby(['Date','Province_State'], as_index=False).agg(agg)
        return df
    
    cols_to_keep = ['UID','iso2', 'iso3',
                    'code3','FIPS','Admin2',
                    'Lat','Long_','Combined_Key',
                    'Province_State','Country_Region']
    c_us = create_df('US_CONFIRMED', 'Confirmed' , {'Confirmed':'sum'}, cols_to_keep)
    
    cols_to_keep = ['UID','iso2', 'iso3',
                    'code3','FIPS','Admin2',
                    'Lat','Long_','Combined_Key',
                    'Province_State','Country_Region','Population'] 
    d_us = create_df('US_DEATHS', 'Deaths' , {'Deaths':'sum','Population':'sum'}, cols_to_keep)
    
    # Merge
    df_us = pd.merge(c_us, d_us, on=['Date','Province_State'], how='left').reset_index(drop=True)
    df_us.loc[(df_us['Deaths'] != 0),'Fatality Rate'] = round((df_us['Deaths']/df_us['Confirmed']*100),2)
    df_us.loc[(df_us['Confirmed'] != 0),'Confirmed Rate'] = round((df_us['Confirmed']/df_us['Population']*100000),0)
    df_us.loc[(df_us['Deaths'] != 0),'Deaths Rate'] = round((df_us['Deaths']/df_us['Population']*100000),0)
    df_us['Date'] = pd.to_datetime(df_us['Date'], format='%m/%d/%y').dt.strftime('%m/%d/%y')
    
    # Check df
    df_check = df_us[(df_us['Date'] == YESTERDAY.strftime('%m/%d/%y')) & 
                (df_us['Population'] != 0) & 
                (df_us['Confirmed'] != 0)]
    
    if df_check.empty:
        print(f'Data not updated !')
        %stop
    else:
        # Generate
        domain_108(df_us)
        df_save(df_us,'US_MAP','csv')
        
    return df_check

df_us = transform_data(check_data())
df_us.head(5)

Dataframe 108 successfully save in database app-wsr in MongoDB. Time: --- 0.049180030822753906 secnds ---
US_MAP successfully saved in US_MAP
CPU times: user 603 ms, sys: 1.53 s, total: 2.14 s
Wall time: 2.37 s


,Date,Province_State,Confirmed,Deaths,Population,Fatality Rate,Confirmed Rate,Deaths Rate
7714,06/11/20,Alabama,22845,755,4903185,3.30,466.0,15.0
7715,06/11/20,Alaska,609,11,731545,1.81,83.0,2.0
7717,06/11/20,Arizona,31267,1135,7278717,3.63,430.0,16.0
7718,06/11/20,Arkansas,10816,171,3017804,1.58,358.0,6.0
7719,06/11/20,California,143377,4934,39512223,3.44,363.0,12.0


In [5]:
def create_map(df,geo,kpi,scale,colors):
    file_path = os.path.join(OUTPUT_FOLDER,f'{kpi}.html')
    fig = px.choropleth_mapbox(df, geojson=geo, color=kpi,
                               locations="Province_State", featureidkey="properties.NAME",
                               range_color=scale,
                               color_continuous_scale=colors,
                               mapbox_style="carto-positron", 
                               zoom=3, center={"lat": 37.0902, "lon": -95.7129},
                               labels={'Province_State':'State'}
                              )
    fig.update_layout(margin={"r":0,"l":0,},autosize=True)
    fig.write_html(file_path)
    updateChartCss(file_path, os.path.join(INPUT_FOLDER,'map.css'))

In [6]:
#Confirmed
kpi='Confirmed'
scale = (100,250000)
colors = [
    [0.0, "#FFFFFF"],
    [0.01,"#B3D7D2"],
    [0.1, "#81B1BD"],
    [0.3, "#437F9D"],
    [1.0, "#00354E"]
]
analysis(df_us, kpi)
create_map(df_us,MAP_STATES_20M,kpi,scale,colors)
file_html = f'{kpi}.html'

#Make html visible
bob_static = bob.makeStaticGettable(os.path.join(OUTPUT_FOLDER,file_html),"dev")
token_conf = bob_static[24:84]
bob_statics = f"{bob_statics}{bob_static}\n"

Confirmed : Min: 30, Max: 380892, Average: 36785.0, AvrMax: 0.1, Med: 16315.0, MedMax: 0.04
[Bob from JupyterHub] => Add this line: 
 # bob.static {"token": "90526ba2d8a80ee6df5572107b610e03eac0d77fb6f821bc1961798e0737", "path": "/home/ftp/ftp/prod/wsr-public/data_output/Confirmed.html", "filename": "../data_output/Confirmed.html"} 

At the top of this file to make it gettable at this url 
 https:/api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/90526ba2d8a80ee6df5572107b610e03eac0d77fb6f821bc1961798e0737 

Now you can remove the code bob.makeStaticGettable() from this file
If you want to remove the gettable capability, just remove the line you have added (# bob.static ...) or the file


In [7]:
#Confirmed Rate
colors = [
    [0.0, "#FFFFFF"],
    [0.01,"#B3D7D2"],
    [0.1, "#81B1BD"],
    [0.3, "#437F9D"],
    [1.0, "#00354E"]
]
scale = (100,1000)
kpi='Confirmed Rate'
analysis(df_us, kpi)
create_map(df_us,MAP_STATES_20M,kpi,scale,colors)
file_html = f'{kpi}.html'

#Make html visible
bob_static = bob.makeStaticGettable(os.path.join(OUTPUT_FOLDER,file_html),"dev")
token_conf_rate = bob_static[24:84]
bob_statics = f"{bob_statics}{bob_static}\n"

Confirmed Rate : Min: 49.0, Max: 1867.0, Average: 524.0, AvrMax: 0.28, Med: 383.0, MedMax: 0.21
[Bob from JupyterHub] => Add this line: 
 # bob.static {"token": "11181c08ecd778256be52dabb087a9219c52fa576bb241dbf45a56fb7355", "path": "/home/ftp/ftp/prod/wsr-public/data_output/Confirmed Rate.html", "filename": "../data_output/Confirmed Rate.html"} 

At the top of this file to make it gettable at this url 
 https:/api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/11181c08ecd778256be52dabb087a9219c52fa576bb241dbf45a56fb7355 

Now you can remove the code bob.makeStaticGettable() from this file
If you want to remove the gettable capability, just remove the line you have added (# bob.static ...) or the file


In [8]:
#'Deaths'
colors = [
    [0.0, "#FFFFFF"],
    [0.01,"#B3D7D2"],
    [0.1, "#81B1BD"],
    [0.3, "#437F9D"],
    [1.0, "#00354E"]
]
scale = (0,10000)
kpi='Deaths'
analysis(df_us, kpi)
create_map(df_us,MAP_STATES_20M,kpi,scale,colors)
file_html = f'{kpi}.html'

#Make html visible
bob_static = bob.makeStaticGettable(os.path.join(OUTPUT_FOLDER,file_html),"dev")
token_death = bob_static[24:84]
bob_statics = f"{bob_statics}{bob_static}\n"

Deaths : Min: 2, Max: 30580, Average: 2069.0, AvrMax: 0.07, Med: 588.0, MedMax: 0.02
[Bob from JupyterHub] => Add this line: 
 # bob.static {"token": "6013feea6a754643b7651e46a86d5f82718c8ef147d0f8c46edb1e9ee0c1", "path": "/home/ftp/ftp/prod/wsr-public/data_output/Deaths.html", "filename": "../data_output/Deaths.html"} 

At the top of this file to make it gettable at this url 
 https:/api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/6013feea6a754643b7651e46a86d5f82718c8ef147d0f8c46edb1e9ee0c1 

Now you can remove the code bob.makeStaticGettable() from this file
If you want to remove the gettable capability, just remove the line you have added (# bob.static ...) or the file


In [9]:
#'Deaths Rate'
colors = [
    [0.0, "#FFFFFF"],
    [0.01,"#B3D7D2"],
    [0.1, "#81B1BD"],
    [0.3, "#437F9D"],
    [1.0, "#00354E"]
]
scale = (10,100)
kpi='Deaths Rate'
analysis(df_us, kpi)
create_map(df_us,MAP_STATES_20M,kpi,scale,colors)
file_html = f'{kpi}.html'

#Make html visible
bob_static = bob.makeStaticGettable(os.path.join(OUTPUT_FOLDER,file_html),"dev")
token_death_rate = bob_static[24:84]
bob_statics = f"{bob_statics}{bob_static}\n"

Deaths Rate : Min: 1.0, Max: 140.0, Average: 26.0, AvrMax: 0.19, Med: 13.0, MedMax: 0.09
[Bob from JupyterHub] => Add this line: 
 # bob.static {"token": "49c17857184e62224756986f99214c1f1fd666544aa0b74c8d8855e1273a", "path": "/home/ftp/ftp/prod/wsr-public/data_output/Deaths Rate.html", "filename": "../data_output/Deaths Rate.html"} 

At the top of this file to make it gettable at this url 
 https:/api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/49c17857184e62224756986f99214c1f1fd666544aa0b74c8d8855e1273a 

Now you can remove the code bob.makeStaticGettable() from this file
If you want to remove the gettable capability, just remove the line you have added (# bob.static ...) or the file


In [10]:
#Fatality Rate
colors = [
    [0.0, "#FFFFFF"],
    [0.25,"#B3D7D2"],
    [0.5, "#81B1BD"],
    [0.75, "#437F9D"],
    [1.0, "#00354E"]
]
scale = (0,10)
kpi='Fatality Rate'
analysis(df_us, kpi)
create_map(df_us,MAP_STATES_20M,kpi,scale,colors)
file_html = f'{kpi}.html'

#Make html visible
bob_static = bob.makeStaticGettable(os.path.join(OUTPUT_FOLDER,file_html),"dev")
token_fatality_rate = bob_static[24:84]
bob_statics = f"{bob_statics}{bob_static}\n"

Fatality Rate : Min: 0.99, Max: 9.33, Average: 4.0, AvrMax: 0.43, Med: 4.0, MedMax: 0.43
[Bob from JupyterHub] => Add this line: 
 # bob.static {"token": "b8009c59b34a427be46a71ef1fb2223e192412136fea205197c94a39a56e", "path": "/home/ftp/ftp/prod/wsr-public/data_output/Fatality Rate.html", "filename": "../data_output/Fatality Rate.html"} 

At the top of this file to make it gettable at this url 
 https:/api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/b8009c59b34a427be46a71ef1fb2223e192412136fea205197c94a39a56e 

Now you can remove the code bob.makeStaticGettable() from this file
If you want to remove the gettable capability, just remove the line you have added (# bob.static ...) or the file


In [11]:
# html
RACINE = 'https://api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/'

CONFIRMED = [{'URL': (RACINE + token_conf),
             'KPI': 'Confirmed',
             'UPPER_FILTER': 'All population'}]

DEATHS = [{'URL': (RACINE + token_death),
           'KPI': 'Deaths',
           'UPPER_FILTER': 'All population'}]

FATALITY_RATE = [{'URL': (RACINE + token_fatality_rate),
            'KPI': 'Fatality Rate',
            'UPPER_FILTER': 'All population'}]

CONFIRMED_RATE = [{'URL': (RACINE + token_conf_rate),
            'KPI': 'Confirmed Ratio',
            'UPPER_FILTER': '100k inhab'}]

DEATH_RATE = [{'URL': (RACINE + token_death_rate),
             'KPI': 'Deaths Ratio',
             'UPPER_FILTER': '100k inhab'}]

#Concat list to create Series
LIST_KPIS = (CONFIRMED + DEATHS + FATALITY_RATE + CONFIRMED_RATE + DEATH_RATE)

#Create Dataframe
DOMAIN = pd.DataFrame(LIST_KPIS)
DOMAIN.loc[:, 'VALUE'] = ("<iframe src='" +
                          DOMAIN['URL'] +
                          ("' frameborder='0' style='overflow:hidden;"
                           "height:85vh;width:100%' height='100%' "
                           "width='100%'></iframe> <style>.modebar"
                           "{display: none!important;}</style>"))
if USE_MONGO:
    bob.mongo.save_df(DOMAIN,'109',DB_APP,True)
DOMAIN

Dataframe 109 successfully save in database app-wsr in MongoDB. Time: --- 0.01955389976501465 secnds ---


,URL,KPI,UPPER_FILTER,VALUE
0,https://api.dev.cashstory.com/api/v1/notebook/...,Confirmed,All population,<iframe src='https://api.dev.cashstory.com/api...
1,https://api.dev.cashstory.com/api/v1/notebook/...,Deaths,All population,<iframe src='https://api.dev.cashstory.com/api...
2,https://api.dev.cashstory.com/api/v1/notebook/...,Fatality Rate,All population,<iframe src='https://api.dev.cashstory.com/api...
3,https://api.dev.cashstory.com/api/v1/notebook/...,Confirmed Ratio,100k inhab,<iframe src='https://api.dev.cashstory.com/api...
4,https://api.dev.cashstory.com/api/v1/notebook/...,Deaths Ratio,100k inhab,<iframe src='https://api.dev.cashstory.com/api...


In [12]:
file = open(os.path.join(OUTPUT_FOLDER,"map_chart_us_statics.ipynb"),'w')
file.write(bob_statics) 
file.close()